[View in Colaboratory](https://colab.research.google.com/github/joaoflf/handson-ml/blob/master/ch11_ex9.ipynb)

In [4]:
!git clone https://github.com/mixuala/colab_utils.git
import os
import colab_utils.tboard
import tensorflow as tf
import numpy as np
from functools import partial

fatal: destination path 'colab_utils' already exists and is not an empty directory.


Implement early stopping

In [5]:
tf.reset_default_graph()
from datetime import datetime
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)
ROOT = %pwd
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=logdir )


n_inputs = 28 * 28
n_hidden1 = 150
n_hidden2 = 150
n_hidden3 = 150
n_hidden4 = 150
n_hidden5 = 150
n_outputs = 5
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')
he_init = tf.contrib.layers.variance_scaling_initializer()

training = tf.placeholder_with_default(False, shape=(), name='training')
dropout = tf.placeholder_with_default(False, shape=(), name='dropout')
batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)

dropout_rate = 0.5
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, name='hidden1', kernel_initializer=he_init)
    bn1 = batch_norm_layer(hidden1)
    bn1_act = tf.nn.elu(bn1)
    bn1_drop = tf.layers.dropout(bn1_act, dropout_rate, training=dropout)
    hidden2 = tf.layers.dense(bn1_drop, n_hidden2, name='hidden2', kernel_initializer= he_init)
    bn2 = batch_norm_layer(hidden2)
    bn2_act = tf.nn.elu(bn2)
    bn2_drop = tf.layers.dropout(bn2_act, dropout_rate, training=dropout)
    hidden3 = tf.layers.dense(bn2_drop, n_hidden3, name='hidden3', kernel_initializer= he_init)
    bn3 = batch_norm_layer(hidden3)
    bn3_act = tf.nn.elu(bn3)
    bn3_drop = tf.layers.dropout(bn3_act, dropout_rate, training=dropout)
    hidden4 = tf.layers.dense(bn3_drop, n_hidden4, name='hidden4', kernel_initializer= he_init)
    bn4 = batch_norm_layer(hidden4)
    bn4_act = tf.nn.elu(bn4)
    bn4_drop = tf.layers.dropout(bn4_act, dropout_rate, training=dropout)
    hidden5 = tf.layers.dense(bn4_drop, n_hidden5, name='hidden5', kernel_initializer= he_init)
    bn5 = batch_norm_layer(hidden5)
    bn5_act = tf.nn.elu(bn5)
    bn5_drop = tf.layers.dropout(bn5_act, dropout_rate, training=dropout)
    logist_before_bn = tf.layers.dense(bn5_drop, n_outputs, name='outputs', kernel_initializer=he_init)
    logits = tf.layers.batch_normalization(logist_before_bn, training=training, momentum=0.9, name='logits')
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss, name="training_op")
    
# Model evalutation
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1, name='correct') #check if the logit with highest value in array is the same as y
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

loss_summary = tf.summary.scalar('Loss', loss)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()
saver = tf.train.Saver()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')

batch_size = 500
n_epochs = 1000
X_train, y_train = mnist.train.images[mnist.train.labels < 5], mnist.train.labels[mnist.train.labels < 5]
X_test, y_test = mnist.test.images[mnist.test.labels < 5], mnist.test.labels[mnist.test.labels < 5]
X_val, y_val = mnist.validation.images[mnist.validation.labels < 5], mnist.validation.labels[mnist.validation.labels < 5]
steps_since_winner = 0
step_limit = 10
best_loss = np.infty

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
import time

start = time.time()


with tf.Session() as sess:
    init.run()
    batch_step = 0
    for epoch in range(n_epochs):
        idx = np.random.permutation(len(X_train))
        for indices in np.array_split(idx, len(idx) // batch_size):
            X_batch, y_batch = X_train[indices], y_train[indices]
            sess.run([training_op, extra_update_ops], feed_dict={training: True, dropout: False, X: X_batch, y: y_batch})
            batch_step+=1
        # Log in tensorboard
        summary_str = loss_summary.eval(feed_dict={X: X_batch, y: y_batch})
        file_writer.add_summary(summary_str,batch_step)
        
        loss_cv, acc_cv = sess.run([loss, accuracy], feed_dict={X: X_val, y: y_val})
        if (best_loss > loss_cv):
            best_loss = loss_cv
            steps_since_winner = 0
            print('Best loss found:  {:.6f}\t accuracy:  {:.6f}'.format(loss_cv, acc_cv))
            save_path = saver.save(sess, './data/best_model_0_4_bn.ckpt')
        else:
            if (steps_since_winner < step_limit):
                steps_since_winner+=1
            else:
                print('Early stopping.')
                break

    file_writer.close()

end = time.time()
print("Total training time: {:.1f}s".format(end - start))
        

ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://3eec106b.ngrok.io
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Best loss found:  0.188237	 accuracy:  0.974199
Best loss found:  0.098237	 accuracy:  0.979672
Best loss found:  0.069003	 accuracy:  0.984754
Best loss found:  0.053068	 accuracy:  0.989445
Best loss found:  0.043713	 accuracy:  0.989836
Best loss found:  0.039273	 accuracy:  0.990618
Best loss found:  0.028934	 accuracy:  0.992181
Best loss found:  0.027530	 accuracy:  0.991790
Best loss found:  0.023650	 accuracy:  0.992963
Best loss found:  0.018847	 accuracy:  0.995700
Best loss found:  0.018096	 accuracy:  0.994136
Best loss found:  0.016125	 accuracy:  0.994527
Best loss found:  0.015750	 accuracy:  0.996091
Best loss found:  0.014733	 accuracy:  0.996091
Best loss found:  0.013334	 accuracy:  0.99

In [20]:
with tf.Session() as sess:

    saver.restore(sess, './data/best_model_0_4_bn.ckpt')
    acc_train = sess.run(accuracy, feed_dict={X: X_train, y: y_train})
    acc_test = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
    print('Without Dropout: train accuracy: {:.2f}\t test accuracy: {:.2f}'.format(acc_train * 100, acc_test * 100))
    


INFO:tensorflow:Restoring parameters from ./data/best_model_0_4_bn.ckpt
Without Dropout: train accuracy: 99.96	 test accuracy: 99.63


Transfer learning

In [3]:
tf.reset_default_graph()

restore_saver = tf.train.import_meta_graph('./data/best_model_0_4_bn.ckpt.meta')

# for op in tf.get_default_graph().get_operations():
#     print(op.name)

X = tf.get_default_graph().get_tensor_by_name('X:0')
y = tf.get_default_graph().get_tensor_by_name('y:0')
training = tf.get_default_graph().get_tensor_by_name('training:0')
dropout = tf.get_default_graph().get_tensor_by_name('dropout:0')
loss = tf.get_default_graph().get_tensor_by_name('loss/loss:0')
accuracy = tf.get_default_graph().get_tensor_by_name('eval/accuracy:0')

# freeze first 5 layers, by minimizing only the last layer variables (logits)
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='hidden5')
optimizer = tf.train.RMSPropOptimizer(learning_rate, name='RMSProp')
training_op = optimizer.minimize(loss, var_list=train_vars, name="training_op")


init = tf.global_variables_initializer()
saver = tf.train.Saver()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')

batch_size = 500
n_epochs = 1000
X_train2, y_train2 = mnist.train.images[mnist.train.labels >= 5], mnist.train.labels[mnist.train.labels >= 5]-5
X_test2, y_test2 = mnist.test.images[mnist.test.labels >= 5], mnist.test.labels[mnist.test.labels >= 5]-5
X_val2, y_val2 = mnist.validation.images[mnist.validation.labels >= 5], mnist.validation.labels[mnist.validation.labels >= 5]-5
steps_since_winner = 0
step_limit = 10
best_loss = np.infty

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
import time

start = time.time()


with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        idx = np.random.permutation(len(X_train2))
        for indices in np.array_split(idx, len(idx) // batch_size):
            X_batch, y_batch = X_train2[indices], y_train2[indices]
            sess.run([training_op, extra_update_ops], feed_dict={training: True, dropout: False, X: X_batch, y: y_batch})
        loss_cv, acc_cv = sess.run([loss, accuracy], feed_dict={X: X_val2, y: y_val2})
        if (best_loss > loss_cv):
            best_loss = loss_cv
            steps_since_winner = 0
            print('Best loss found:  {:.6f}\t accuracy:  {:.6f}'.format(loss_cv, acc_cv))
            save_path = saver.save(sess, './data/best_model_5_9_bn.ckpt')
        else:
            if (steps_since_winner < step_limit):
                steps_since_winner+=1
            else:
                print('Early stopping.')
                break

    file_writer.close()

end = time.time()
print("Total training time: {:.1f}s".format(end - start))


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Best loss found:  0.917658	 accuracy:  0.711302
Best loss found:  0.584950	 accuracy:  0.865684
Best loss found:  0.509098	 accuracy:  0.881654
Best loss found:  0.499879	 accuracy:  0.899263
Best loss found:  0.466927	 accuracy:  0.904177
Best loss found:  0.449320	 accuracy:  0.911548
Best loss found:  0.428723	 accuracy:  0.914005
Best loss found:  0.420993	 accuracy:  0.920557
Best loss found:  0.411393	 accuracy:  0.921785
Best loss found:  0.404247	 accuracy:  0.925471
Best loss found:  0.382775	 accuracy:  0.929156
Best loss found:  0.368496	 accuracy:  0.938575
Best loss found:  0.365889	 accuracy:  0.933251
Best loss found:  0.361836	 accuracy:  0.935708
Best loss found:  0.361646	 accuracy:  0.933251
Best loss found:  0.349121	 accuracy:  0.936937
Best loss found:  0.346553	 accuracy:  0.9

In [22]:
with tf.Session() as sess:

    saver.restore(sess, './data/best_model_5_9_bn.ckpt')
    acc_train = sess.run(accuracy, feed_dict={X: X_train2, y: y_train2})
    acc_test = sess.run(accuracy, feed_dict={X: X_test2, y: y_test2})
    print('Without Dropout: train accuracy: {:.2f}\t test accuracy: {:.2f}'.format(acc_train * 100, acc_test * 100))
    

INFO:tensorflow:Restoring parameters from ./data/best_model_5_9_bn.ckpt
Without Dropout: train accuracy: 94.13	 test accuracy: 93.44
